In [ ]:
# Loading Dataset

import pandas as pd
df = pd.read_csv("../DataSets/Transport_AssignmentsRAW.csv")
# Transport_AssignmentsRAW
# df.head(5)
# print(df.head(2))
# DIsssSdata
df.info()

In [ ]:
# Convert string columns to datetime
df['Expiry_Date'] = pd.to_datetime(df['Expiry_Date'], format='%d/%m/%Y')
df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%d/%m/%Y')

# Calculate expiry range
df['Expiry_Days_Left'] = (df['Expiry_Date'] - df['Order_Date']).dt.days

# Display
print(df[['OrderID', 'Order_Date', 'Expiry_Date', 'Expiry_Days_Left']].head(5))


Initial shape: (197184, 27)


,Assignment_ID,Product_ID,Urgent_Flag,Dispatch_Window,Delivery_Window,Unit_Weight_(kg),Unit_Volume_(L),Fragile_Flag,Temp_Sensitive_Flag,Quantity_Assigned,...,Vehicle_Type_x,Capacity_Type_x,Volume_Capacity_(L)_x,Weight_Capacity_(kg)_x,Vehicle_ID_y,Assigned_Vehicles,Vehicle_Type_y,Capacity_Type_y,Volume_Capacity_(L)_y,Weight_Capacity_(kg)_y
0,ZONE-3::RACK-1::854::GJ27X6555,TOP-1005-B43-854,1,2025-07-04,2025-07-07,1.8,1.0,1,0,27,...,Specialised,Sp.Van,15000,4000,GJ17X4217,"GJ17X4217, GJ18X2499, GJ22X5898, GJ11X9382, GJ...",General,Truck,35000,9000
1,ZONE-3::RACK-1::854::GJ27X6555,TOP-1005-B43-854,1,2025-07-04,2025-07-07,1.8,1.0,1,0,27,...,Specialised,Sp.Van,15000,4000,GJ17X4217,"GJ17X4217, GJ18X2499, GJ22X5898, GJ11X9382, GJ...",Specialised,Sp.Van,15000,4000
2,ZONE-3::RACK-1::854::GJ27X6555,TOP-1005-B43-854,1,2025-07-04,2025-07-07,1.8,1.0,1,0,27,...,Specialised,Sp.Van,15000,4000,GJ18X2499,"GJ17X4217, GJ18X2499, GJ22X5898, GJ11X9382, GJ...",General,Truck,35000,9000
3,ZONE-3::RACK-1::854::GJ27X6555,TOP-1005-B43-854,1,2025-07-04,2025-07-07,1.8,1.0,1,0,27,...,Specialised,Sp.Van,15000,4000,GJ18X2499,"GJ17X4217, GJ18X2499, GJ22X5898, GJ11X9382, GJ...",Specialised,Sp.Van,15000,4000
4,ZONE-3::RACK-1::854::GJ27X6555,TOP-1005-B43-854,1,2025-07-04,2025-07-07,1.8,1.0,1,0,27,...,Specialised,Sp.Van,15000,4000,GJ22X5898,"GJ17X4217, GJ18X2499, GJ22X5898, GJ11X9382, GJ...",General,Truck,35000,9000


In [ ]:
# Reorder columns (make Expiry_Days_Left 2nd)
cols = df.columns.tolist()
cols.insert(1, cols.pop(cols.index('Expiry_Days_Left')))
df = df[cols]

# Save back to CSV
df.to_csv("../DataSets/Transport_AssignmentsMOD.csv", index=False)

Shape after dropping invalid dates: (197184, 29)


In [ ]:
df.describe()
df.info()

Final dataset shape: (0, 29)


In [ ]:
#Loading Required Liaberies ( WORKING WITH RANDOM FOREST KNN HANDELED BY SCIKIT LEARN )

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib  # For saving model


Label Mapping:


In [ ]:
#Loading DataSheet
df = pd.read_csv("../DataSets/Transport_AssignmentsMOD.csv")
df.head(3)

,OrderID,Expiry_Days_Left,Order_Date,Expiry_Date,Pallet_Size,Route_Type,Mode_of_Transport,Urgent_Order_Flag,Fragility,Dispatch_Window,Temp_Sensitive
0,D0689266-88BE-4949-B7A4,238,2025-06-15,2026-02-08,80,Interstate,Big Truck,Yes,No,26/06/2025,Yes
1,85E9D4B1-BB58-4F2D-86BA,143,2025-06-18,2025-11-08,120,Interstate,Big Truck,Yes,Yes,23/06/2025,No
2,DA6CC913-1DC3-4E6A-B78E,127,2025-06-20,2025-10-25,150,Interstate,Big Truck,No,Yes,23/06/2025,Yes


In [ ]:
# df['Expiry_Date'] = pd.to_datetime(df['Expiry_Date'], format='%Y-%m-%d')
# df['Dispatch_Window'] = pd.to_datetime(df['Dispatch_Window'], format='%Y-%m-%d')
# df['Order_Date'] = pd.to_datetime(df['Order_Date'], format='%Y-%m-%d')

# Convert Dispatch_Window and Order_Date to delivery_window_days
df['Dispatch_Window'] = pd.to_datetime(df['Dispatch_Window'])
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Delivery_Window_Days'] = (df['Dispatch_Window'] - df['Order_Date']).dt.days


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21588\771286952.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Dispatch_Window'] = pd.to_datetime(df['Dispatch_Window'])


In [ ]:
import numpy as np

def assign_priority_fuzzy_sigmoid_robust(row,
                                         expiry_threshold=120,          # 2 months
                                         delivery_threshold=10,         # 10 days
                                         high_cutoff=0.6,
                                         medium_cutoff=0.4,
                                         weights=None,
                                         k=0.08):                       # Sharper sigmoid
    """
    Robust industrial fuzzy logic for assigning priority labels to dispatch items.
    Combines rule-based overrides and smooth sigmoid-based scoring.

    Parameters:
        expiry_threshold (int): Days threshold where expiry becomes critical
        delivery_threshold (int): Days threshold for delivery urgency
        high_cutoff (float): Score threshold for High priority
        medium_cutoff (float): Score threshold for Medium priority
        weights (dict): Weight mapping for all five parameters
        k (float): Sigmoid steepness factor (higher = sharper cutoff)

    Returns:
        str: 'High', 'Medium', or 'Low'
    """
    if weights is None:
        weights = {
            'expiry': 0.4,
            'delivery': 0.3,
            'urgent': 0.2,
            'temp': 0.05,
            'fragile': 0.05
        }

    expiry_days = row['Expiry_Days_Left']
    delivery_days = row['Delivery_Window_Days']

    # 🧠 Sigmoid fuzzy scores (scaled to smooth transitions)
    expiry_score = 1 / (1 + np.exp(k * (expiry_days - expiry_threshold)))
    delivery_score = 1 / (1 + np.exp(k * (delivery_days - delivery_threshold)))

    urgent_score = 1 if row['Urgent_Order_Flag'] == 'Yes' else 0
    temp_score = 1 if row['Temp_Sensitive'] == 'Yes' else 0
    fragile_score = 1 if row['Fragility'] == 'Yes' else 0

    # 🔒 Hard Rule Overrides (Business-critical triggers)
    # Rule 1: Expiry very soon AND urgent
    if expiry_days <= 30 and row['Urgent_Order_Flag'] == 'Yes':
        return 'High'

    # Rule 2: Temp-sensitive AND near expiry
    if expiry_days <= 30 and row['Temp_Sensitive'] == 'Yes':
        return 'High'

    # Rule 3: Tight delivery window AND urgent
    if delivery_days <= 5 and row['Urgent_Order_Flag'] == 'Yes':
        return 'High'

    # Rule 4: Expiry + Delivery both extremely tight
    if expiry_days <= 10 and delivery_days <= 5:
        return 'High'

    # Rule 5: Fragile and urgent (special handling needed)
    if row['Urgent_Order_Flag'] == 'Yes' and row['Fragility'] == 'Yes':
        return 'High'

    # Rule 6: Very relaxed, non-urgent case
    if expiry_days > 600 and delivery_days > 45 and row['Urgent_Order_Flag'] == 'No':
        return 'Low'

    # 🎯 Weighted fuzzy scoring
    total_score = (
        expiry_score * weights['expiry'] +
        delivery_score * weights['delivery'] +
        urgent_score * weights['urgent'] +
        temp_score * weights['temp'] +
        fragile_score * weights['fragile']
    )

    # 🔽 Priority Classification
    if total_score >= high_cutoff:
        return 'High'
    elif total_score >= medium_cutoff:
        return 'Medium'
    else:
        return 'Low'



df['Priority_Label'] = df.apply(assign_priority_fuzzy_sigmoid_robust, axis=1)

# Split data
high_df = df[df['Priority_Label'] == 'High']
medium_df = df[df['Priority_Label'] == 'Medium']
low_df = df[df['Priority_Label'] == 'Low']

print(f"High Priority Orders: {len(high_df)}")
print(f"Medium Priority Orders: {len(medium_df)}")
print(f"Low Priority Orders: {len(low_df)}")


High Priority Orders: 678
Medium Priority Orders: 202
Low Priority Orders: 1120


In [ ]:
#dataset for ml

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Step 1: Feature Selection
features = [
    'Pallet_Size',
    'Route_Type',
    'Mode_of_Transport',
    'Urgent_Order_Flag',
    'Fragility',
    'Temp_Sensitive',
    'Expiry_Days_Left',
    'Dispatch_Window',  # will convert this to days
    'Order_Date'
]

# Convert Dispatch_Window and Order_Date to delivery_window_days
df['Dispatch_Window'] = pd.to_datetime(df['Dispatch_Window'])
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Delivery_Window_Days'] = (df['Dispatch_Window'] - df['Order_Date']).dt.days

# Keep only required features + label
ml_df = df[[
    'Pallet_Size', 'Route_Type', 'Mode_of_Transport', 'Urgent_Order_Flag',
    'Fragility', 'Temp_Sensitive', 'Expiry_Days_Left', 'Delivery_Window_Days', 
    'Priority_Label'
]]

# Encode categorical columns
label_encoders = {}
for col in ['Route_Type', 'Mode_of_Transport', 'Urgent_Order_Flag', 'Fragility', 'Temp_Sensitive']:
    le = LabelEncoder()
    ml_df.loc[:, col] = le.fit_transform(ml_df[col])
    label_encoders[col] = le

# Encode output label
le_priority = LabelEncoder()
# ml_df.loc[:, 'Priority_Label'] = le_priority.fit_transform(ml_df['Priority_Label'])
ml_df.loc[:, 'Priority_Label'] = le_priority.fit_transform(ml_df['Priority_Label'])
# ml_df['Priority_Label'] = le_priority.fit_transform(ml_df['Priority_Label'].astype(str))



In [37]:
#BRUT FORCE

# Manually map string labels to integers
priority_mapping = {
    'Low': 0,
    'Medium': 1,
    'High': 2
}

# Apply the mapping manually
df['Encoded_Priority_Label'] = df['Priority_Label'].map(priority_mapping)

# Check results
print(df['Encoded_Priority_Label'].unique())
print(df['Encoded_Priority_Label'].dtype)


[0 2 1]
int64


In [38]:
# Ensure encoded label is added to ml_df from df
ml_df['Encoded_Priority_Label'] = df['Encoded_Priority_Label']

# Define features (X) and label (y)
X = ml_df.drop(['Priority_Label', 'Encoded_Priority_Label'], axis=1)
y = ml_df['Encoded_Priority_Label']

# Split into train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_5292\2616058964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_df['Encoded_Priority_Label'] = df['Encoded_Priority_Label']


In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le_priority.classes_))




Confusion Matrix:
 [[223   1   0]
 [  3  37   0]
 [  0   0 136]]

Classification Report:
               precision    recall  f1-score   support

        High       0.99      1.00      0.99       224
         Low       0.97      0.93      0.95        40
      Medium       1.00      1.00      1.00       136

    accuracy                           0.99       400
   macro avg       0.99      0.97      0.98       400
weighted avg       0.99      0.99      0.99       400



In [ ]:
print(df['Priority_Label'].value_counts())
print(df['Priority_Label'].isnull().sum())


Priority_Label
Low       1120
High       678
Medium     202
Name: count, dtype: int64
0


In [41]:
#SEPERATE DATAFRAMES FOR PRIORITY

high_df = df[df['Priority_Label'] == 'High']
medium_df = df[df['Priority_Label'] == 'Medium']
low_df = df[df['Priority_Label'] == 'Low']

 
print(f"High Priority Orders: {len(high_df)}")
print(f"Medium Priority Orders: {len(medium_df)}")
print(f"Low Priority Orders: {len(low_df)}")

High Priority Orders: 678
Medium Priority Orders: 202
Low Priority Orders: 1120


In [42]:
# Save model
joblib.dump(model, '..\\TrainedModel\\dispatch_priority_model.pkl')

# # Also save encoders if needed later
joblib.dump(label_encoders, '..\\TrainedModel\\label_encoders.pkl')
joblib.dump(le_priority, '..\\TrainedModel\\priority_label_encoder.pkl')

['..\\TrainedModel\\priority_label_encoder.pkl']

In [43]:
# 3 MODEL USING AND HYPER PARAMETERS 

for col, le in label_encoders.items():
    print(f" Test Data Format ::: {col} classes: {le.classes_}")

 Test Data Format ::: Route_Type classes: ['Intercity' 'Interstate' 'Local']
 Test Data Format ::: Mode_of_Transport classes: ['Big Truck' 'Medium Truck' 'Small Truck']
 Test Data Format ::: Urgent_Order_Flag classes: ['No' 'Yes']
 Test Data Format ::: Fragility classes: ['No' 'Yes']
 Test Data Format ::: Temp_Sensitive classes: ['No' 'Yes']


In [ ]:
import joblib
import pandas as pd

# Load model and encoders
model = joblib.load('..\\TrainedModel\\dispatch_priority_model.pkl')
label_encoders = joblib.load('..\\TrainedModel\\label_encoders.pkl')
le_priority = joblib.load('..\\TrainedModel\\priority_label_encoder.pkl')

# Test data with possible new/unseen values
new_data = pd.DataFrame([
    {
        'Pallet_Size': 100,
        'Route_Type': 'Intercity',        # OK
        'Mode_of_Transport': 'Medium Truck',  # OK
        'Urgent_Order_Flag': 'Yes',
        'Fragility': 'Yes',
        'Temp_Sensitive': 'Yes',
        'Expiry_Days_Left': 5,
        'Delivery_Window_Days': 10
    },
    {
        'Pallet_Size': 80,
        'Route_Type': 'Intracity',
        'Mode_of_Transport': 'Mini Truck',
        'Urgent_Order_Flag': 'No',
        'Fragility': 'Yes',
        'Temp_Sensitive': 'No',
        'Expiry_Days_Left': 90,
        'Delivery_Window_Days': 25
    },
    {
        'Pallet_Size': 120,
        'Route_Type': 'Interstate',
        'Mode_of_Transport': 'Big Truck',
        'Urgent_Order_Flag': 'No',
        'Fragility': 'No',
        'Temp_Sensitive': 'No',
        'Expiry_Days_Left': 360,
        'Delivery_Window_Days': 60
    }
])

# Encode new data safely by replacing unseen labels
for col in ['Route_Type', 'Mode_of_Transport', 'Urgent_Order_Flag', 'Fragility', 'Temp_Sensitive']:
    le = label_encoders[col]
    
    # Ensure all values are known to encoder
    new_data[col] = new_data[col].apply(lambda x: x if x in le.classes_ else le.classes_[0])
    
    # Re-transform with safe values
    new_data[col] = le.transform(new_data[col])

# Predict
prediction_encoded = model.predict(new_data)
prediction_label = le_priority.inverse_transform(prediction_encoded)

# Results
new_data['Predicted_Priority'] = prediction_label
print(new_data[['Pallet_Size', 'Expiry_Days_Left', 'Delivery_Window_Days', 'Predicted_Priority']])


   Pallet_Size  Expiry_Days_Left  Delivery_Window_Days Predicted_Priority
0          100                 5                    10             Medium
1           80                90                    25                Low
2          120               360                    60               High


In [45]:
# Assuming 'df' contains the final dataframe after priority assignment
csv_path = "../DataSets/Transport_AssignmentsMLGEN.csv"
df.to_csv(csv_path, index=False)